## DSP-634 - Master targeting lists for CENTERSTONE
* source: https://achievements.atlassian.net/browse/DSP-634
* Criterion:
  * Age 18-64
  * Must reside in any of the zip codes listed in column D on this sheet.


In [23]:
import pandas as pd

In [24]:
list_zipcode_file = pd.read_csv("/repos/amezlini_Draft/Zipcodes_CENTERSTONE_Study.tsv", sep = '\t',skiprows=1)

In [25]:
list_zipcode = sum([sub.split(",") for sub in list_zipcode_file['Zip Codes Within 30 miles']],[])
print('N =', len(list_zipcode))
print('N =', len(pd.unique(list_zipcode)))

N = 4324
N = 3390


In [26]:
%load_ext watermark
%watermark -v -p pandas

Python implementation: CPython
Python version       : 3.7.4
IPython version      : 7.14.0

pandas: 1.2.0



### Load survey snapshot for given date

In [27]:
SELECT_DATE = '2021-02-02'

In [28]:
df_demog = pd.read_parquet(f'/mnt/data/survey_snapshots/{SELECT_DATE}_health_survey.parquet')
print(df_demog.shape)
print('N =', df_demog.user_id.nunique())
print(df_demog.columns)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/survey_snapshots/2021-02-02_health_survey.parquet'

#### Only 29K of the 1.2M live in the selected zipcodes

In [8]:
(df_demog.zipcode.isin(list_zipcode)).sum()

29176

#### Only 425 out of 549 provided zipcodes are in survey 

In [9]:
df_demog.zipcode[df_demog.zipcode.isin(list_zipcode)].nunique()

425

### Age 18 - 64 -- use `groupby` to catch multiple entries per UID

In [11]:
list_uid = list(set(df_demog.query('(age >= 18) & (age <=64) & (zipcode == @list_zipcode)')['user_id']))
print('Age 18-64 & zipcode match, N = ', len(list_uid))

Age 18-64 & zipcode match, N =  28804


### Save results to `/mnt/artifacts/results` for later download

In [12]:
pd.DataFrame(list_uid, columns=['user_id']).to_csv('/mnt/artifacts/results/dsp598_01_centerstone_targeting_list_zc90017_n28804.csv', index=None)